In [1]:
!pip install transformers

     |████████████████████████████████| 890kB 2.8MB/s 
     |████████████████████████████████| 890kB 18.2MB/s 
     |████████████████████████████████| 3.0MB 19.4MB/s 
     |████████████████████████████████| 1.1MB 31.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=11d78e8d5c4383a29059852962a7b84729b650528d7a87a0b9cddabc57308488
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [12]:
import torch

In [5]:
from transformers import BertTokenizer, BertModel

In [6]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states = True, # Whether the model returns all hidden-states.
                                  )

# Put the model in "evaluation" mode, meaning feed-forward operation.
model.eval()

### 3 sentences

In [8]:
text1= "What’s the time now in Singapore?"
text2= "What is the weather in Seattle today?"
text3= "Apple is looking at buying the U.K. startup for $1 billion."

In [75]:
def get_sent_embedding(text):
    # Add the special tokens.
    marked_text = "[CLS] " + text + " [SEP]"

    # Split the sentence into tokens.
    tokenized_text = tokenizer.tokenize(marked_text)

    # Map the token strings to their vocabulary indeces.
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(tokenized_text)

    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
        hidden_states = outputs[2]

    token_vecs = hidden_states[-4][0]

    sentence_embedding = torch.mean(token_vecs, dim=0)

    return sentence_embedding


In [76]:
sentence_embedding_text1 = get_sent_embedding(text1)
sentence_embedding_text2 = get_sent_embedding(text2)
sentence_embedding_text3 = get_sent_embedding(text3)

In [78]:
from scipy.spatial.distance import cosine


In [79]:
simi_1vs2 = 1 - cosine(sentence_embedding_text1, sentence_embedding_text2)
simi_1vs3 = 1 - cosine(sentence_embedding_text1, sentence_embedding_text3)
simi_2vs3 = 1 - cosine(sentence_embedding_text2, sentence_embedding_text3)

In [83]:
print(f"Similarity between sentence 1 to sentence 2 is  {simi_1vs2}")
print(f"Similarity between sentence 1 to sentence 3 is  {simi_1vs3}")
print(f"Similarity between sentence 2 to sentence 3 is  {simi_2vs3}")

Similarity between sentence 1 to sentence 2 is  0.8359468579292297
Similarity between sentence 1 to sentence 3 is  0.5442742109298706
Similarity between sentence 2 to sentence 3 is  0.5663244724273682
